In [8]:
import pandas as pd

In [9]:
import sys
sys.path.insert(0,"../scoring")

In [10]:
from scoring_bhc import calculate_scores_bhc
from scoring_bhc import compute_overall_score_bhc

In [11]:
from scoring_instructions import calculate_scores_instructions
from scoring_instructions import compute_overall_score_instructions

# Test

In [12]:
# notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/comparison_simple_gpt3.5_2shot_v2.csv")
notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/submission.csv")

In [13]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [14]:
discharge_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge.csv.gz", keep_default_na=False)


In [15]:
notes = notes.merge(discharge_test[['hadm_id',"text"]], on="hadm_id")

In [38]:
notes

,Unnamed: 0,hadm_id,discharge_instruction_gpt,Original,text
0,0,26231944,You were admitted to the hospital for decompen...,"Dear Mr. ___,\n\nIt was a pleasure to care for...",\nName: ___ Unit No: ___\n ...
1,1,28542384,You were admitted to the hospital for a recent...,"Dear Ms. ___, \n We had the pleasure of taking...",\nName: ___ Unit No: ___\n \nA...
2,2,26489329,You were admitted to the hospital for a possib...,You were admitted to the hospital after a with...,\nName: ___ Unit No: ___\n ...
3,3,22641254,You were admitted to the hospital for hypoxemi...,"Dear Ms. ___, \n\nIt was a pleasure caring for...",\nName: ___ Unit No: ___\n ...
4,4,29265750,You were admitted to the hospital for right-si...,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",\nName: ___ Unit No: ...
...,...,...,...,...,...
14692,14692,27445071,"Dear Mr. or Ms. [Patient's Last Name], You wer...","Dear Mr. ___,\n\nYou were admitted after getti...",\nName: ___ Unit No: ___\...
14693,14693,26192891,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___,\n\n You were admitted to ___ af...",\nName: ___ Unit No: ___\n ...
14694,14694,26584893,"Dear Ms. ___, You were admitted to the hospita...","Dear Ms. ___,\n\nIt was a pleasure taking ___ ...",\nName: ___ Unit No: _...
14695,14695,25623241,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___, \n\nYou were admitted to ___ \nb...",\nName: ___ Unit No: ___\n ...


In [39]:
# hadm_id,brief_hospital_course,discharge_instructions

In [40]:
reference = notes[['hadm_id', "Original"]].rename({"Original":"brief_hospital_course"}, axis=1)
generated = notes[['hadm_id', "discharge_instruction_gpt"]].rename({"discharge_instruction_gpt":"brief_hospital_course"}, axis=1)

In [36]:
notes.isna().sum(), generated.isna().sum()

(Unnamed: 0                   0
 hadm_id                      0
 discharge_instruction_gpt    1
 Original                     0
 text                         0
 dtype: int64,
 hadm_id                  0
 brief_hospital_course    1
 dtype: int64)

In [43]:
# looks like hadm_id 28358078 didn't get generated, and I think it throws an error, so we're dropping it for now
generated = generated[generated['hadm_id'] != 28358078]
reference = reference[reference['hadm_id'] != 28358078]


In [44]:
scores = calculate_scores_bhc(generated,
                 reference,
                 [
                     "bleu", 
                     "rouge", 
                     "bertscore", 
                     "meteor"
                 ])

Beginning scoring...
bleuScorer initialized
rougeScorer initialized
bertScorer initialized


[nltk_data] Downloading package wordnet to /home/vs428/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vs428/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vs428/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteorScorer initialized
Processed 128/14696 samples.
Processed 256/14696 samples.
Processed 384/14696 samples.
Processed 512/14696 samples.
Processed 640/14696 samples.
Processed 768/14696 samples.
Processed 896/14696 samples.
Processed 1024/14696 samples.
Processed 1152/14696 samples.
Processed 1280/14696 samples.
Processed 1408/14696 samples.
Processed 1536/14696 samples.
Processed 1664/14696 samples.
Processed 1792/14696 samples.
Processed 1920/14696 samples.
Processed 2048/14696 samples.
Processed 2176/14696 samples.
Processed 2304/14696 samples.
Processed 2432/14696 samples.
Processed 2560/14696 samples.
Processed 2688/14696 samples.
Processed 2816/14696 samples.
Processed 2944/14696 samples.
Processed 3072/14696 samples.
Processed 3200/14696 samples.
Processed 3328/14696 samples.
Processed 3456/14696 samples.
Processed 3584/14696 samples.
Processed 3712/14696 samples.
Processed 3840/14696 samples.
Processed 3968/14696 samples.
Processed 4096/14696 samples.
Processed 4224/14696 s

In [45]:
compute_overall_score_bhc(scores)

Computing overall score...
Done.


{'bleu': 0.025703420659624474,
 'rouge1': 0.33952083578709863,
 'rouge2': 0.08486844444470257,
 'rougel': 0.17360548057604078,
 'bertscore': 0.2870371798924518,
 'meteor': 0.2756587994626359,
 'overall': 0.19773236013709236}